In [22]:
def read_fasta(file_name):
    '''
    arguments:
    - file_name: path to input FASTA file, type=str

    returns:
    - header: input FASTA header, type=str
    - sequence: input FASTA sequence, type=str
    '''
    sequence = ""

    with open(file_name, 'r') as f_fasta:
        header = f_fasta.readline()

        for line in f_fasta:
            sequence += line.replace('\n', '')
    
    return header, sequence

In [23]:
def find_struct(pos, struct_pos):
    '''
    arguments:
    - pos: genomic position, type=int
    - struct_pos: list of ALPL-specific genomic structures (exons/introns),
      with elements: (structure_name, start, end)

    returns:
    - struct: structure_name (e.g., "exon1"), type=str
    - start: start position of genomic structure, type=int
    
    '''
    for struct, start, end in struct_pos:
        if start <= pos <= end:
            return struct, start

In [24]:
def replace_variant(sequence, position, variant, start_pos_struct):
    '''
    arguments:
    - sequence: genomic sequence, type=str
    - position: genomic position to replace with variant, type=int
    - variant: type=str
    - start_pos_struct: start position of genomic structure (exon/intron), type=int

    - returns: genomic sequence with variant, type=str
    '''
    seq_pos = position - start_pos_struct

    new_sequence = sequence[:seq_pos] + variant + sequence[seq_pos+1:]

    assert len(new_sequence) == len(sequence)

    return new_sequence

In [34]:
def write_fasta(file_name, header, sequence):
    '''
    arguments:
    - file_name: type=str,
    - header: output FASTA header, type=str,
    - sequence: output FASTA sequence, type=str
    '''
    with open(file_name, 'w+') as f:
        f.write(header)
        f.write(sequence)

In [35]:
# list of ALPL-specific genomic structures:
# (exon/intron, start, end)
struct_pos = [
    ("exon1", 21509323, 21509617),
    ("exon2", 21553878, 21554242),
    ("exon3", 21560526, 21560845),
    ("exon4", 21560997, 21561312),
    ("exon5", 21563010, 21563384),
    ("exon6", 21563941, 21564316),
    ("exon7", 21568004, 21568347),
    ("exon8", 21570205, 21570474),
    ("exon9", 21573565, 21573899),
    ("exon10", 21575633, 21576024),
    ("exon11", 21576422, 21576741),
    ("exon12", 21577283, 21578510),
    ("intron1", 21509418, 21554077),
    ("intron2", 21554043, 21560725),
    ("intron3", 21560646, 21561196),
    ("intron4", 21561113, 21563209),
    ("intron5", 21563185, 21564140),
    ("intron6", 21564117, 21568203),
    ("intron7", 21568148, 21570404),
    ("intron8", 21570275, 21573764),
    ("intron9", 21573700, 21575832),
    ("intron10", 21575825, 21576621),
    ("intron11", 21576542, 21577482)
]

### Parse ClinVar results for the ALPL gene

In [36]:
'''
Take ClinVar results trimmed to columns:

[Name, Gene(s), Accession, GRCh38Location, Variant type]

Filter to include only SNVs.

Save to dict with key=variantID, values=[genomic position, variant].
'''

var_ids = []
variants_dict = {}

with open("../data/clinvar_result_trimmed.txt") as f_clinvar:
    # skip header
    next(f_clinvar)

    for line in f_clinvar:
        split_line = line.rstrip().split('\t')

        (name, gene, var_id, location, var_type) = split_line

        # skip name if not NM_000478 (ALPL RefSeqID)
        if "NM_000478" not in name:
            continue
        # extract SNV
        split_name = name.split(' ')
        var_nt = split_name[0][-1]

        # skip non ALPL-specific variants
        if gene != "ALPL":
            continue

        # keep only SNVs for now
        if var_type != "single nucleotide variant":
            continue

        else:
            var_ids.append(var_id)
            variants_dict[var_id] = [int(location), var_nt]

### Create a FASTA file per intron / exon and variant

In [37]:
'''
Take dict var_ids with key=variantID, values=[genomic position, variant]

For each variant (SNV), create a FASTA file (per exon/intron) with sequence containing the variant
'''

for var_id in var_ids:
    # get variant position and nucleotide
    variant_pos = variants_dict[var_id][0]
    variant_nt = variants_dict[var_id][1]

    # find genomic structure (exon, intron) and its starting position for the variant
    struct, start_pos = find_struct(pos=variant_pos, struct_pos=struct_pos)

    # read genomic structure FASTA
    input_file = f"../data/genomic_structures/ALPL_{struct}.fasta"
    header, sequence = read_fasta(input_file)

    # replace variant in sequence of genomic structure
    new_sequence = replace_variant(sequence, position=variant_pos, variant=variant_nt, start_pos_struct=start_pos)

    # save genomic structure FASTA with variant
    output_fasta = f"../data/genomic_structures_variants/ALPL_{struct}_{var_id}.fasta"
    write_fasta(output_fasta, header, new_sequence)